In [67]:
import numpy as np
import time
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score

In [68]:
AND_gate=pd.DataFrame({"A":[0,0,1,1],
                      "B":[0,1,0,1],
                      "Z":[0,0,0,1]})
XOR_gate=pd.DataFrame({"A":[0,0,1,1],
                      "B":[0,1,0,1],
                      "Z":[0,1,1,0]})

In [69]:
class ANN:
    
    def __init__(self,X,Y,h,h1,learning_rate,activation_function,iterations,verbose=False):
        
        self.X = X
        self.Y = Y
        self.h = h
        self.h1 = h1
        self.learning_rate = learning_rate
        self.iterations = iterations
        self.verbose = verbose
        self.activation_function=activation_function
        
        #in this model iam using 2 hidden layers
        
    def layer_sizes(self,X,Y):
        input_layer_size=X.shape[0]
        hidden_layer_size=self.h
        hidden_layer_size1=self.h1
        output_layer_size=Y.shape[0]
        return (input_layer_size,hidden_layer_size,hidden_layer_size1,output_layer_size)
    
    def initialize_parameters(self,i,o):
        W1 = np.random.randn(self.h,i)*0.01
        b1 = np.zeros((self.h,1))
        W2 = np.random.randn(self.h1,self.h)*0.01
        b2 = np.zeros((self.h1,1))
        W3 = np.random.randn(o,self.h1)*0.01
        b3 = np.zeros((o,1))
        parameters = {"W1": W1,
                      "b1": b1,
                      "W2": W2,
                      "b2": b2,
                      "W3": W3,
                      "b3": b3}
        return parameters
    
    def forward_propagation(self,X,parameters):
    
        #load the parameters
        W1 = parameters['W1']
        b1 = parameters['b1']
        W2 = parameters['W2']
        b2 = parameters['b2']
        W3 = parameters["W3"]
        b3 = parameters["b3"]

        #compute Z and A values
        
        if self.activation_function=="tanh":
            Z1 = np.dot(W1,X)+b1
            A1 = np.tanh(Z1)
            Z2 = np.dot(W2,A1)+b2
            A2 = np.tanh(Z2)
            Z3 = np.dot(W3,A2)+b3
            A3=1/(1+np.exp(-Z3))
            cache = {"Z1": Z1,
                     "A1": A1,
                     "Z2": Z2,
                     "A2": A2,
                     "Z3": Z3,
                     "A3": A3}
            
        if self.activation_function=="sigmoid":
            Z1 = np.dot(W1,X)+b1
            A1 = 1/(1+np.exp(-Z1))
            Z2 = np.dot(W2,A1)+b2
            A2 = 1/(1+np.exp(-Z2))
            Z3 = np.dot(W3,A2)+b3
            A3=1/(1+np.exp(-Z3))
            cache = {"Z1": Z1,
                     "A1": A1,
                     "Z2": Z2,
                     "A2": A2,
                     "Z3": Z3,
                     "A3": A3}

        return A3,cache
    
    def compute_cost(self,A3,Y,parameters):
    
        m=Y.shape[1]

        cost=-np.sum((np.dot(np.log(A3),Y.T)+np.dot(np.log(1-A3),(1-Y).T)))/m

        cost=float(np.squeeze(cost))

        return cost
    
    def backward_propagation(self,X,Y,cache,parameters):
    
        m=X.shape[1]

        W1=parameters["W1"]
        W2=parameters["W2"]
        W3=parameters["W3"]

        A1 = cache["A1"]
        A2 = cache["A2"]
        A3 = cache["A3"]
        
    
        #dz(n)=(w(n+1).dz(n+1))*(1-f'(x)^2)
        #deltaW(n)= (dz(n).A(n-1))/m
        if self.activation_function=="tanh":
            dZ3=np.subtract(A3,Y)
            dW3=np.dot(dZ3,A2.T)
            db3=(np.sum(dZ3,axis=1,keepdims=True))/m
            dZ2=np.dot(W3.T,dZ3)*(1-np.power(A2,2))
            dW2=np.dot(dZ2,A1.T)/m
            db2=(np.sum(dZ2,axis=1,keepdims=True))/m
            dZ1=np.dot(W2.T,dZ2)*(1-np.power(A1,2))
            dW1=np.dot(dZ1,X.T)/m
            db1=(np.sum(dZ1,axis=1,keepdims=True))/m

            grads = {"dW1": dW1,
                     "db1": db1,
                     "dW2": dW2,
                     "db2": db2,
                     "dW3": dW3,
                     "db3": db3}
            
        if self.activation_function=="sigmoid":
            dZ3=np.subtract(A3,Y)
            dW3=np.dot(dZ3,A2.T)
            db3=(np.sum(dZ3,axis=1,keepdims=True))/m
            dZ2=np.dot(W3.T,dZ3)*(1-A2)*A2
            dW2=np.dot(dZ2,A1.T)/m
            db2=(np.sum(dZ2,axis=1,keepdims=True))/m
            dZ1=np.dot(W2.T,dZ2)*(1-A1)*A1
            dW1=np.dot(dZ1,X.T)/m
            db1=(np.sum(dZ1,axis=1,keepdims=True))/m

            grads = {"dW1": dW1,
                     "db1": db1,
                     "dW2": dW2,
                     "db2": db2,
                     "dW3": dW3,
                     "db3": db3}
    
        return grads

    def update_parameters(self,parameters,grads,lr):
    
        W1 = parameters["W1"]
        b1 = parameters["b1"]
        W2 = parameters["W2"]
        b2 = parameters["b2"]
        W3 = parameters["W3"]
        b3 = parameters["b3"]

        dW1 = grads["dW1"]
        db1 = grads["db1"]
        dW2 = grads["dW2"]
        db2 = grads["db2"]
        dW3 = grads["dW3"]
        db3 = grads["db3"]

        W1 = W1-lr*dW1
        b1 = b1-lr*db1
        W2 = W2-lr*dW2
        b2 = b2-lr*db2
        W3 = W3-lr*dW3
        b3 = b3-lr*db3

        parameters = {"W1": W1,
                      "b1": b1,
                      "W2": W2,
                      "b2": b2,
                      "W3": W3,
                      "b3": b3}
        return parameters
    
    def process(self):
        
        #np.random.seed(3)
        i=self.layer_sizes(self.X,self.Y)[0]
        o=self.layer_sizes(self.X,self.Y)[3]
        #print(i,o)
        self.parameters=self.initialize_parameters(i=i,o=o)
        #print(self.parameters)
        self.cost_error=[]
        
        for j in range(self.iterations):
                
            self.A3,self.cache=self.forward_propagation(self.X,self.parameters)
            
            cost=self.compute_cost(self.A3,self.Y,self.parameters)
            
            self.grads=self.backward_propagation(self.X,self.Y,self.cache,self.parameters)
            
            self.parameters=self.update_parameters(self.parameters,self.grads,self.learning_rate)
            
            s=1000
            if self.verbose and (j%s==0):
                y_pred=self.predictions(X)
                acc=accuracy_score(self.Y[0],y_pred)*100
                self.cost_error.append(cost)
                print(f"cost at {j} iteration is ====> {cost} and accuracy score is {acc}")

        return self.cost_error
    
    def predictions(self,X):
            A3,cache=self.forward_propagation(X,self.parameters)
            predictions = []
            for i in range(len(A3[0])):
                if (A3[0][i]>0.5).any():
                    predictions.append(1)
                else:
                    predictions.append(0)
            predictions=np.array(predictions)
            return predictions
    
    def hypothesis(self,X,Y):
        x_min,x_max=x[:,0].min()-1,x[:,0].max()+1
        y_min,y_max =x[:,1].min()-1,x[:, 1].max()+1
        xx,yy=np.meshgrid(np.arange(x_min,x_max,0.1),
                             np.arange(x_min,x_max,0.1))
        z=self.predictions(np.c_[xx.ravel(),yy.ravel()].T)
        z=z.reshape(xx.shape)
        plt.contourf(xx,yy,z,alpha=0.4)
        plt.scatter(x[:,0],x[:,1],c=y,s=20,edgecolor='k')
        plt.show()
        

In [70]:
X=np.array(AND_gate.drop(["Z"],axis=1))
Y=np.array(AND_gate["Z"])
X=X.reshape(X.shape[1],X.shape[0])
Y=Y.reshape(1,Y.shape[0])
model=ANN(X,Y,h=3,h1=2,learning_rate=0.05,activation_function="sigmoid",iterations=10000,verbose=True)
c=model.process()
y_pred=model.predictions(X)

cost at 0 iteration is ====> 0.6921273705457 and accuracy score is 75.0
cost at 1000 iteration is ====> 0.562317587594888 and accuracy score is 75.0
cost at 2000 iteration is ====> 0.5622703671426512 and accuracy score is 75.0
cost at 3000 iteration is ====> 0.5621101929978813 and accuracy score is 75.0
cost at 4000 iteration is ====> 0.5615581571298199 and accuracy score is 75.0
cost at 5000 iteration is ====> 0.5595529165816473 and accuracy score is 75.0
cost at 6000 iteration is ====> 0.5506158412805172 and accuracy score is 75.0
cost at 7000 iteration is ====> 0.47811079414966196 and accuracy score is 75.0
cost at 8000 iteration is ====> 0.10321805068423082 and accuracy score is 100.0
cost at 9000 iteration is ====> 0.029985542119306183 and accuracy score is 100.0


In [71]:
X=np.array(XOR_gate.drop(["Z"],axis=1))
Y=np.array(XOR_gate["Z"])
X=X.reshape(X.shape[1],X.shape[0])
Y=Y.reshape(1,Y.shape[0])
model=ANN(X,Y,h=3,h1=2,learning_rate=0.05,activation_function="sigmoid",iterations=10000,verbose=True)
c=model.process()
y_pred=model.predictions(X)

cost at 0 iteration is ====> 0.6931477551921927 and accuracy score is 50.0
cost at 1000 iteration is ====> 0.6931471826390081 and accuracy score is 50.0
cost at 2000 iteration is ====> 0.6931471824891937 and accuracy score is 50.0
cost at 3000 iteration is ====> 0.6931471823415324 and accuracy score is 50.0
cost at 4000 iteration is ====> 0.6931471821958963 and accuracy score is 50.0
cost at 5000 iteration is ====> 0.693147182052162 and accuracy score is 50.0
cost at 6000 iteration is ====> 0.6931471819102083 and accuracy score is 50.0
cost at 7000 iteration is ====> 0.6931471817699172 and accuracy score is 50.0
cost at 8000 iteration is ====> 0.6931471816311736 and accuracy score is 50.0
cost at 9000 iteration is ====> 0.6931471814938648 and accuracy score is 50.0


In [72]:
from sklearn.neural_network import MLPClassifier
import pandas as pd

In [73]:
model=MLPClassifier(solver='lbfgs',alpha=1e-5,
                    hidden_layer_sizes=(8,3,1))

In [3]:
AND_gate=pd.DataFrame({"A":[0,0,1,1],
                      "B":[0,1,0,1],
                      "Z":[0,0,0,1]})
XOR_gate=pd.DataFrame({"A":[0,0,1,1],
                      "B":[0,1,0,1],
                      "Z":[0,1,1,0]})

X=np.array(AND_gate.drop(["Z"],axis=1))
Y=np.array(AND_gate["Z"])
print(AND_gate)

   A  B  Z
0  0  0  0
1  0  1  0
2  1  0  0
3  1  1  1


In [75]:
model.fit(X,Y)

MLPClassifier(alpha=1e-05, hidden_layer_sizes=(8, 3, 1), solver='lbfgs')

In [76]:
model.predict(X)

array([0, 0, 0, 0], dtype=int64)

In [77]:
Y

array([0, 0, 0, 1], dtype=int64)

In [2]:
import numpy as np
import random
import sympy as sp
import pandas as pd

In [18]:
s=pd.read_excel("C:\\Users\\vigne\\Desktop\\df2.xlsx")
print(s.head())
df=s[['Kilometers_Driven']]
df1=s[['Mileage','Engine',"Seats","New_Price"]]
df1= df1[['Mileage','Engine',"New_Price"]].replace(to_replace=r'([a-z/]+|[A-Z/]+)', value='', regex=True)
df2=pd.concat([df,s[['Seats']]],axis=1, join='inner')
df2=pd.concat([df2,df1],axis=1, join='inner')
df2['Mileage'] = df2['Mileage'].astype(float)
df2['Engine'] = df2['Engine'].astype(float)
df2['New_Price'] = df2['New_Price'].astype(float)
df2.head()

   Unnamed: 0  Unnamed: 0.1                                       Name  \
0           0             2  Toyota Innova Crysta Touring Sport 2.4 MT   
1           1            30                 Toyota Etios Cross 1.4L VD   
2           2            36               Skoda Rapid 1.6 MPI AT Style   
3           3            39              BMW 5 Series 520d Luxury Line   
4           4            43               Skoda Rapid 1.6 MPI Ambition   

  Location  Year  Kilometers_Driven Fuel_Type Transmission Owner_Type  \
0   Mumbai  2017              34000    Diesel       Manual      First   
1  Chennai  2014              70000    Diesel       Manual     Second   
2   Mumbai  2017              13500    Petrol    Automatic      First   
3   Mumbai  2012              66889    Diesel    Automatic      First   
4   Jaipur  2018               4000    Petrol       Manual      First   

      Mileage   Engine       Power  Seats   New_Price  
0  13.68 kmpl  2393 CC   147.8 bhp      7  25.27 Lakh  
1  2

,Kilometers_Driven,Seats,Mileage,Engine,New_Price
0,34000,7,13.68,2393.0,25.27
1,70000,5,23.59,1364.0,9.27
2,13500,5,14.84,1598.0,14.95
3,66889,5,22.48,1995.0,70.43
4,4000,5,15.41,1598.0,11.89
